## Load reservoir operation results conducted by Jia

This script loads Rdata of the hydropower operation results and save as
python preferred format.

In [1]:
import numpy as np
import pandas as pd
import os
os.environ['R_HOME'] = '/Users/dlee/anaconda3/envs/dlee/lib/R'  # The R path can be found by $ R RHOME
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
import rpy2.robjects as robjects
pandas2ri.activate()

# Load Dam Inflow data from SUTD
dfFlowDams = pd.read_hdf('./data/dfFlowDams.hdf')
ind_dams = np.load('./data/ind_dams.npz')['ind_dams']
damList = dfFlowDams.columns.values

# Load Degree of Regulation (DOR)
dor = pd.read_hdf('./data/new_dor.hdf')      # The order is sorted during the process...
dorx = dor.loc[dor.DOR2 <= 0, 'GRAND_ID']
dorx_ind = np.isin(damList, dorx.values)

In [2]:
# Load dam operation results data
rf = r['load']('./data/rdata/simulation_2020_MP.Rdata')
rf = r['simulation_2020_MP']

# Total hydropower productiona
prod = np.zeros([4, damList.shape[0]])      # [dp, sdp_t, mpc_pf, mpc_df3]
for i in range(damList.shape[0]):
    prod[0,i] = pandas2ri.ri2py_floatvector(rf[i][0][0][6])     # DP
    prod[1,i] = np.sum(pandas2ri.ri2py_floatvector(rf[i][1][3][5])[:501])*(365.25/12*24)    # SDP_t
    prod[2,i] = pandas2ri.ri2py_floatvector(rf[i][2][0][6])     # MPC_PF
    prod[3,i] = pandas2ri.ri2py_floatvector(rf[i][3][0][6])     # MPC_DF3
prod = pd.DataFrame(prod, ['dp', 'sdp_t', 'mpc_pf', 'mpc_df3'], damList)
prod.columns.name = 'GRAND_ID'
prod = prod.transpose()
# Calculate improvements
prod['PF'] = (prod['mpc_pf'] - prod['sdp_t'])/prod['mpc_pf']*100
prod['DF'] = (prod['mpc_df3'] - prod['sdp_t'])/prod['mpc_pf']*100

if True:
    filn_prod = './data/hydropower_production.hdf'
    prod.to_hdf(filn_prod, key='df', complib='blosc:zstd', complevel=9)
    print('%s is saved.' % filn_prod)

./data/hydropower_production.hdf is saved.


In [5]:
# Validation of data
# Load hydropower production results
dfProd = pd.read_hdf('./data/hydropower_production.hdf')
dfProd_dor = dfProd[np.isin(dfProd.index, dorx)]

# Check with Jia's results: 1593 (48.4%) and 735 (31.7%)
superior = np.sum(np.round(dfProd['mpc_df3'],5) > np.round(dfProd['sdp_t'],5))/len(dfProd)
assert round(superior, 4) == 0.4884
superior_dor = np.sum(np.round(dfProd_dor['mpc_df3'],5) > np.round(dfProd_dor['sdp_t'],5))/len(dfProd_dor)
assert round(superior_dor, 4) == 0.3170

### Load regression variables

In [6]:
# Load 58 variables used for regression
fn = './data/rdata/data_for_regression_2019.Rdata'
rf = r['load'](fn)
variable = r['data']
variable.index = damList
variable.index.name = 'GRAND_ID'
# Save the DataFrame
filn_vari = './data/regression_variables.hdf'
variable.to_hdf(filn_vari, key='df', complib='blosc:zstd', complevel=9)
print('%s is saved.' % filn_vari)
variable.head()

/Users/dlee/anaconda3/envs/dlee_sovi/lib/python3.6/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


./data/regression_variables.hdf is saved.


,DAM_HGT,AREA,CAP,CAPLIVE_used,ECAP,FLOW_M3S,HEAD,MAXDEP_used,CATCH,dep_over_head,...,rel_over_inf_med,drift_rel_mon,drift_rel_year,drift2_rel_mon,drift2_rel_year,period_lt_qmax,spill_over_rel,spill_over_inf,zero_storage,head_over_max
GRAND_ID,,,,,,,,,,,,,,,,,,,,,
6,186.0,1761.000000,74300.0,74289.624051,2730.0,1932.985166,159.964130,184.0,71707.0,1.150258,...,1.069414,0.055708,0.235012,1.821737,7.862010,1.000000,0.010694,0.010377,0.000000,0.989244
24,38.0,27.585558,320.7,320.700000,47.0,127.363220,41.796730,36.0,1403.0,0.861311,...,0.900715,0.216573,0.910227,1.536084,6.333463,0.731755,0.138458,0.121349,0.001972,0.917783
25,53.0,63.000000,1002.8,1002.800000,65.0,115.831412,63.558771,51.0,1161.0,0.802407,...,0.976611,0.087138,0.366309,1.665518,6.877380,0.771203,0.049523,0.047061,0.000000,0.962114
27,17.0,120.100000,1620.0,1620.000000,46.0,72.509272,71.854301,15.0,1251.0,0.208756,...,0.918933,0.144954,0.604518,1.677994,6.694755,0.207101,0.061929,0.057004,0.000000,0.972298
31,30.0,6.209736,77.0,77.000000,28.0,27.214487,116.532310,28.0,234.0,0.240277,...,1.044707,0.149846,0.687509,1.321027,5.904540,1.000000,0.114650,0.102970,0.009862,0.965686
